In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import re
warnings.filterwarnings('ignore')
# Machine learning algorithms.
from collections import Counter
import lightgbm as lgb
import xgboost as xgb
from sklearn import metrics
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 10
import unicodedata

In [2]:
#Loading data and creating a copy of test dataframe
train=pd.read_csv(r"E:\evaluation\p13\Final_Train.csv",encoding='latin1')
test=pd.read_csv(r"E:\evaluation\p13\Final_Test.csv",encoding='latin1')
df_test = test.copy()

In [3]:
#Lets check the first 5 data
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250


In [4]:
#Lets check for rows and columns in train dataset
train.shape

(5961, 7)

In [5]:
#Lets check for rows and columns in test dataset
test.shape

(1987, 6)

In [6]:
#Lets check the first 5 data
test.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info
0,MBBS,35 years experience,NaN,"Ghatkopar East, Mumbai",General Medicine,NaN
1,"MBBS, Diploma in Otorhinolaryngology (DLO)",31 years experience,NaN,"West Marredpally, Hyderabad",ENT Specialist,NaN
2,"MBBS, DDVL",40 years experience,70%,"KK Nagar, Chennai",Dermatologists,"70% 4 Feedback KK Nagar, Chennai"
3,BAMS,0 years experience,NaN,"New Ashok Nagar, Delhi",Ayurveda,NaN
4,"BDS, MDS - Conservative Dentistry & Endodontics",16 years experience,100%,"Kanakpura Road, Bangalore",Dentist,General Dentistry Conservative Dentistry Cosme...


In [7]:
#Lets verify train dataset datatype and null values 
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Qualification       5961 non-null   object
 1   Experience          5961 non-null   object
 2   Rating              2659 non-null   object
 3   Place               5936 non-null   object
 4   Profile             5961 non-null   object
 5   Miscellaneous_Info  3341 non-null   object
 6   Fees                5961 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 326.1+ KB


In [9]:
train.isnull().sum()
#So rating , place and miscellaneous info have null values

Qualification            0
Experience               0
Rating                3302
Place                   25
Profile                  0
Miscellaneous_Info    2620
Fees                     0
dtype: int64

In [10]:
# check train column types

ctype = train.dtypes.reset_index()
ctype.columns = ["Count", "Column Type"]
ctype.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,int64,1
1,object,6


In [11]:
# check test column types

ctype = test.dtypes.reset_index()
ctype.columns = ["Count", "Column Type"]
ctype.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,object,6


In [12]:
# Train set for minimum and maximum qualification 
dat_train = train.Qualification.apply(lambda x: len(x.split(',')))
print("Maximum qualifications of a doctor in the Train dataset is {}\n".format(dat_train.max()))
print("And the qualifications is --> {}\n\n".format(train.Qualification[dat_train.idxmax()]))
print("Minimum qualification of a doctor in the Train dataset is {}\n".format(dat_train.min()))
print("And the qualifications is --> {}\n\n".format(train.Qualification[dat_train.idxmin()]))
# Test set for minimum and maximum qualification 
dat_test = test.Qualification.apply(lambda x: len(x.split(',')))
print("Maximum qualifications of a doctor in the Test dataset is {}\n".format(dat_test.max()))
print("And the qualifications is --> {}\n\n".format(test.Qualification[dat_test.idxmax()]))
print("Minimum qualification of a doctor in the Test dataset is {}\n".format(dat_test.min()))
print("And the qualifications is --> {}".format(test.Qualification[dat_test.idxmin()]))

Maximum qualifications of a doctor in the Train dataset is 10

And the qualifications is --> Fellowship in Oral implantlogy, Certified Advance Course In Endodontics, Certified in Orthodontics, Certification in Full Mouth Rehabilitation, Certified BPS Dentist, Certificate in Cosmetic Dentistry, Professional Implantology Training Course (PITC), Fellowship in Advanced Endoscopic Sinus Surgery, Degree in Dental Implant, Fellowship in Lasers & Cosmetology


Minimum qualification of a doctor in the Train dataset is 1

And the qualifications is --> BAMS


Maximum qualifications of a doctor in the Test dataset is 17

And the qualifications is --> BDS, PG Diploma in Conservative, Endodontics and Aesthetic Dentistry, Fellowship In Clinical Cosmetology, PG Diploma In Clinical Cosmetology (PGDCC), MBA - Hospital Management, Diploma in Cosmetic Dentistry, Fellow of Academy of General Education (FAGE), Advanced Retreatment Course in RCT, Advanced Endotontic Course On Root Canal, Certified Course in 

In [13]:
#Split qualification with , 
sorted(test.Qualification[test.Qualification.apply(lambda x: len(x.split(','))).idxmax()].split(","))

[' Advanced Endotontic Course On Root Canal',
 ' Advanced Retreatment Course in RCT',
 ' Certificate in Cosmetic Dentistry',
 ' Certification in Prosthodontics & Periodontology',
 ' Certified Advanced Course in Wisdom Tooth Extraction',
 ' Certified Course in Oral Surgery',
 ' Diploma in Cosmetic Dentistry',
 ' Endodontics and Aesthetic Dentistry',
 ' Fellow of Academy of General Education (FAGE)',
 ' Fellowship In Clinical Cosmetology',
 ' Fellowship in Aesthetic Dentistry (FAD)',
 ' MBA - Hospital Management',
 ' MIDA',
 ' PG Diploma In Clinical Cosmetology (PGDCC)',
 ' PG Diploma in Conservative',
 ' Post Graduate Certificate in Endodontics (PGCE)',
 'BDS']

In [14]:
# Define function to remove inconsistencies in the data
def sortQual(text):
    arr = re.sub(r'\([^()]+\)', lambda x: x.group().replace(",","-"), text) # to replace ',' with '-' inside brackets only
    return ','.join(sorted(arr.lower().replace(" ","").split(",")))

In [15]:
# Apply the function on the Qualification set

# Train Set
train.Qualification = train.Qualification.apply(lambda x: sortQual(x))

# Test Set
test.Qualification = test.Qualification.apply(lambda x: sortQual(x))

In [16]:
# Define a function to create a doc of all Qualifications seprataed by ','

def doc(series):
    Quals = ''
    for i in series:
        Quals += i + ','
    return Quals

In [17]:
# List of top 10 unique Qualifications along with there occurence in Train Set

text = doc(train.Qualification)
df = pd.DataFrame.from_dict(dict(Counter(text.split(',')).most_common()), orient='index').reset_index()
df.columns=['Qualification','Count']
df.head(10)

,Qualification,Count
0,mbbs,2808
1,bds,1363
2,bams,764
3,bhms,749
4,md-dermatology,606
5,ms-ent,411
6,venereology&leprosy,297
7,md-generalmedicine,285
8,diplomainotorhinolaryngology(dlo),250
9,md-homeopathy,181


In [18]:
# List of top 10 unique Qualifications along with there occurence in Test Set

text = doc(test.Qualification)
df = pd.DataFrame.from_dict(dict(Counter(text.split(',')).most_common()), orient='index').reset_index()
df.columns=['Qualification','Count']
df.head(10)

,Qualification,Count
0,mbbs,980
1,bds,428
2,bhms,249
3,bams,243
4,md-dermatology,196
5,ms-ent,128
6,md-generalmedicine,99
7,venereology&leprosy,91
8,diplomainotorhinolaryngology(dlo),85
9,ddvl,67


In [19]:

text = doc(test.Qualification)
df = pd.DataFrame.from_dict(dict(Counter(text.split(',')).most_common()), orient='index').reset_index()
df.columns=['Qualification','Count']
df['code'] = df.Qualification.astype('category').cat.codes
df.head(10)


,Qualification,Count,code
0,mbbs,980,268
1,bds,428,14
2,bhms,249,16
3,bams,243,12
4,md-dermatology,196,286
5,ms-ent,128,354
6,md-generalmedicine,99,290
7,venereology&leprosy,91,425
8,diplomainotorhinolaryngology(dlo),85,112
9,ddvl,67,68


In [20]:
qual_dict = dict(zip(df.Qualification, df.code))

In [21]:
def qual_col(dataframe, col, col_num):
    return dataframe[col].str.split(',').str[col_num]

In [22]:
# for training set
for i in range(0,dat_train.max()):
    qual = "Qual_"+ str(i+1)
    train[qual] = qual_col(train,'Qualification', i)

    
# for test set
for i in range(0,dat_test.max()):
    qual = "Qual_"+ str(i+1)
    test[qual] = qual_col(test,'Qualification', i)

In [23]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10
0,"bhms,md-homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"bams,md-ayurvedamedicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"mbbs,ms-otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"bams,bsc-zoology",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bams,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Train set
train['years_exp'] = train['Experience'].str.slice(stop=2).astype(int)

# Test set
test['years_exp'] = test['Experience'].str.slice(stop=2).astype(int)

In [25]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,years_exp
0,"bhms,md-homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
1,"bams,md-ayurvedamedicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,"mbbs,ms-otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
3,"bams,bsc-zoology",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
4,bams,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20


In [26]:
# Train set
train['Rating'].fillna('0%',inplace = True)
train['Rating'] = train['Rating'].str.slice(stop=-1).astype(int)

# Test set
test['Rating'].fillna('0%',inplace = True)
test['Rating'] = test['Rating'].str.slice(stop=-1).astype(int)

In [27]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,years_exp
0,"bhms,md-homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
1,"bams,md-ayurvedamedicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,"mbbs,ms-otorhinolaryngology",9 years experience,0,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
3,"bams,bsc-zoology",12 years experience,0,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
4,bams,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20


In [28]:
 #Train Set
train['City'] = train['Place'].str.split(',').str[1]
train['Locality'] = train['Place'].str.split(',').str[0]


# Test Set
test['City'] = test['Place'].str.split(',').str[1]
test['Locality'] = test['Place'].str.split(',').str[0]

In [29]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,years_exp,City,Locality
0,"bhms,md-homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Ernakulam,Kakkanad
1,"bams,md-ayurvedamedicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Whitefield
2,"mbbs,ms-otorhinolaryngology",9 years experience,0,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,Bangalore,Mathikere - BEL
3,"bams,bsc-zoology",12 years experience,0,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Bannerghatta Road
4,bams,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,Chennai,Keelkattalai


In [30]:
list(train.Miscellaneous_Info[0:10])

['100% 16 Feedback Kakkanad, Ernakulam',
 '98% 76 Feedback Whitefield, Bangalore',
 nan,
 'Bannerghatta Road, Bangalore ?250 Available on Sun, 10 Feb',
 '100% 4 Feedback Keelkattalai, Chennai',
 nan,
 nan,
 'Dental Fillings Crowns and Bridges Fixing Impaction / Impacted Tooth Extraction',
 nan,
 nan]

In [31]:
# Train set
train.Miscellaneous_Info = train.Miscellaneous_Info.str.replace(unicodedata.lookup('Indian Rupee Sign'), 'INR ')

# Test set
test.Miscellaneous_Info = test.Miscellaneous_Info.str.replace(unicodedata.lookup('Indian Rupee Sign'), 'INR ')

In [32]:
list(train.Miscellaneous_Info[0:10])

['100% 16 Feedback Kakkanad, Ernakulam',
 '98% 76 Feedback Whitefield, Bangalore',
 nan,
 'Bannerghatta Road, Bangalore ?250 Available on Sun, 10 Feb',
 '100% 4 Feedback Keelkattalai, Chennai',
 nan,
 nan,
 'Dental Fillings Crowns and Bridges Fixing Impaction / Impacted Tooth Extraction',
 nan,
 nan]

In [33]:
# Define function to return the Feedback numbers

def find_feedback(data):
    result = re.search(r' (.*?) Feedback',data)
    if result:
        return int(result.group(1))
    else:
        return 0

In [34]:
# Fetch out the feedback numbers in different records. 

# Train set
train['feedack_num'] = train.Miscellaneous_Info.apply(lambda x: find_feedback(x) if '%' in str(x) else 0)

# Test set
test['feedack_num'] = test.Miscellaneous_Info.apply(lambda x: find_feedback(x) if '%' in str(x) else 0)

In [35]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,...,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,years_exp,City,Locality,feedack_num
0,"bhms,md-homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,24,Ernakulam,Kakkanad,16
1,"bams,md-ayurvedamedicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Whitefield,76
2,"mbbs,ms-otorhinolaryngology",9 years experience,0,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9,Bangalore,Mathikere - BEL,0
3,"bams,bsc-zoology",12 years experience,0,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Bannerghatta Road,0
4,bams,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20,Chennai,Keelkattalai,4


In [36]:
# Define function to return the Fees Value

def find_fees(data):
    result = re.search(r'INR (\d*)',data)
    if result:
        return int(result.group(1))
    else:
        return 0

In [37]:
# Fetch out the Fees value in different records. 

# Train set
train['fees_val'] = train.Miscellaneous_Info.apply(lambda x: find_fees(x) if 'INR' in str(x) else 0)

# Test set
test['fees_val'] = test.Miscellaneous_Info.apply(lambda x: find_fees(x) if 'INR' in str(x) else 0)

In [38]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,...,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,years_exp,City,Locality,feedack_num,fees_val
0,"bhms,md-homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,...,NaN,NaN,NaN,NaN,NaN,24,Ernakulam,Kakkanad,16,0
1,"bams,md-ayurvedamedicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,...,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Whitefield,76,0
2,"mbbs,ms-otorhinolaryngology",9 years experience,0,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,...,NaN,NaN,NaN,NaN,NaN,9,Bangalore,Mathikere - BEL,0,0
3,"bams,bsc-zoology",12 years experience,0,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,...,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Bannerghatta Road,0,0
4,bams,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20,Chennai,Keelkattalai,4,0


In [39]:
# Select Qualification categorical columns to be encoded

column_test = ['Qual_1', 'Qual_2', 'Qual_3', 'Qual_4',
           'Qual_5', 'Qual_6', 'Qual_7', 'Qual_8', 'Qual_9', 'Qual_10', 'Qual_11',
           'Qual_12', 'Qual_13', 'Qual_14', 'Qual_15', 'Qual_16', 'Qual_17']

column_train = ['Qual_1', 'Qual_2', 'Qual_3', 'Qual_4',
           'Qual_5', 'Qual_6', 'Qual_7', 'Qual_8', 'Qual_9', 'Qual_10']

In [40]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,...,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,years_exp,City,Locality,feedack_num,fees_val
0,"bhms,md-homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,...,NaN,NaN,NaN,NaN,NaN,24,Ernakulam,Kakkanad,16,0
1,"bams,md-ayurvedamedicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,...,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Whitefield,76,0
2,"mbbs,ms-otorhinolaryngology",9 years experience,0,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,...,NaN,NaN,NaN,NaN,NaN,9,Bangalore,Mathikere - BEL,0,0
3,"bams,bsc-zoology",12 years experience,0,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,...,NaN,NaN,NaN,NaN,NaN,12,Bangalore,Bannerghatta Road,0,0
4,bams,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20,Chennai,Keelkattalai,4,0


In [41]:
test.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Qual_1,Qual_2,Qual_3,Qual_4,...,Qual_13,Qual_14,Qual_15,Qual_16,Qual_17,years_exp,City,Locality,feedack_num,fees_val
0,mbbs,35 years experience,0,"Ghatkopar East, Mumbai",General Medicine,NaN,mbbs,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,35,Mumbai,Ghatkopar East,0,0
1,"diplomainotorhinolaryngology(dlo),mbbs",31 years experience,0,"West Marredpally, Hyderabad",ENT Specialist,NaN,diplomainotorhinolaryngology(dlo),mbbs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,31,Hyderabad,West Marredpally,0,0
2,"ddvl,mbbs",40 years experience,70,"KK Nagar, Chennai",Dermatologists,"70% 4 Feedback KK Nagar, Chennai",ddvl,mbbs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,40,Chennai,KK Nagar,4,0
3,bams,0 years experience,0,"New Ashok Nagar, Delhi",Ayurveda,NaN,bams,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,Delhi,New Ashok Nagar,0,0
4,"bds,mds-conservativedentistry&endodontics",16 years experience,100,"Kanakpura Road, Bangalore",Dentist,General Dentistry Conservative Dentistry Cosme...,bds,mds-conservativedentistry&endodontics,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16,Bangalore,Kanakpura Road,0,0


In [42]:
# Define function to label encode the selected categorical variable for modeling

def encode(data):
    return data.astype('category').cat.codes

In [43]:
# Encode categorical column of test data

columns = ['Profile','City','Locality']

for i in columns:
    col = i+"_code"
    test[col] = encode(test[i])

In [44]:
test.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Qual_1,Qual_2,Qual_3,Qual_4,...,Qual_16,Qual_17,years_exp,City,Locality,feedack_num,fees_val,Profile_code,City_code,Locality_code
0,mbbs,35 years experience,0,"Ghatkopar East, Mumbai",General Medicine,NaN,mbbs,NaN,NaN,NaN,...,NaN,NaN,35,Mumbai,Ghatkopar East,0,0,4,6,142
1,"diplomainotorhinolaryngology(dlo),mbbs",31 years experience,0,"West Marredpally, Hyderabad",ENT Specialist,NaN,diplomainotorhinolaryngology(dlo),mbbs,NaN,NaN,...,NaN,NaN,31,Hyderabad,West Marredpally,0,0,3,5,577
2,"ddvl,mbbs",40 years experience,70,"KK Nagar, Chennai",Dermatologists,"70% 4 Feedback KK Nagar, Chennai",ddvl,mbbs,NaN,NaN,...,NaN,NaN,40,Chennai,KK Nagar,4,0,2,1,212
3,bams,0 years experience,0,"New Ashok Nagar, Delhi",Ayurveda,NaN,bams,NaN,NaN,NaN,...,NaN,NaN,0,Delhi,New Ashok Nagar,0,0,0,3,373
4,"bds,mds-conservativedentistry&endodontics",16 years experience,100,"Kanakpura Road, Bangalore",Dentist,General Dentistry Conservative Dentistry Cosme...,bds,mds-conservativedentistry&endodontics,NaN,NaN,...,NaN,NaN,16,Bangalore,Kanakpura Road,0,0,1,0,231


In [45]:
# Create unique lists of [variable, variable code] combination and drop duplicate pairs.

df_test_merge_1 = test[['Profile','Profile_code']].drop_duplicates()
df_test_merge_2 = test[['City','City_code']].drop_duplicates()
df_test_merge_3 = test[['Locality','Locality_code']].drop_duplicates()

In [46]:
# Pull the respective encoded variables list in the train data (Using a left join) to avoid any merging issue.

train = pd.merge(train,df_test_merge_1[['Profile','Profile_code']],on='Profile', how='left')
train = pd.merge(train,df_test_merge_2[['City','City_code']],on='City', how='left')
train = pd.merge(train,df_test_merge_3[['Locality','Locality_code']],on='Locality', how='left')

In [47]:
# Train set after merging encoded categories

train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Qual_1,Qual_2,Qual_3,...,Qual_9,Qual_10,years_exp,City,Locality,feedack_num,fees_val,Profile_code,City_code,Locality_code
0,"bhms,md-homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,bhms,md-homeopathy,NaN,...,NaN,NaN,24,Ernakulam,Kakkanad,16,0,5,4.0,221.0
1,"bams,md-ayurvedamedicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,bams,md-ayurvedamedicine,NaN,...,NaN,NaN,12,Bangalore,Whitefield,76,0,0,0.0,579.0
2,"mbbs,ms-otorhinolaryngology",9 years experience,0,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,mbbs,ms-otorhinolaryngology,NaN,...,NaN,NaN,9,Bangalore,Mathikere - BEL,0,0,3,0.0,320.0
3,"bams,bsc-zoology",12 years experience,0,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ?250 Available on...",250,bams,bsc-zoology,NaN,...,NaN,NaN,12,Bangalore,Bannerghatta Road,0,0,0,0.0,56.0
4,bams,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,bams,NaN,NaN,...,NaN,NaN,20,Chennai,Keelkattalai,4,0,0,1.0,249.0


In [48]:
def missing_values_table(df):
# Total missing values
        mis_val = df.isnull().sum()
        
 # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
 # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
 # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
# Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
 # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
 # Return the dataframe with missing information
        return mis_val_table_ren_columns

missing_values_table(train)

Your selected dataframe has 25 columns.
There are 15 columns that have missing values.


,Missing Values,% of Total Values
Qual_10,5960,100.0
Qual_9,5957,99.9
Qual_8,5956,99.9
Qual_7,5949,99.8
Qual_6,5928,99.4
Qual_5,5845,98.1
Qual_4,5589,93.8
Qual_3,4591,77.0
Miscellaneous_Info,2620,44.0
Qual_2,1962,32.9


In [50]:
cols_to_use = ['Qual_1', 'Qual_2', 'Qual_3', 'Qual_4',
               'Qual_5', 'Qual_6', 'Qual_7', 'Qual_8',
               'Qual_9', 'Qual_10', 'Profile_code', 'City_code', 'Locality_code',
               'feedack_num', 'fees_val','years_exp','Rating']

target_col = 'Fees'

In [51]:
for i in cols_to_use:
    train[i] = pd.to_numeric(train[i].astype(str).str.replace(',',''), errors='coerce').fillna(-1).astype(int)

In [52]:
train.fillna(-1, inplace=True)

In [53]:
# Define LGBM function

def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        'metric': 'rmse',
        "objective" : "regression",
        "boosting": "gbdt",
        "random_state": 2019,
        "learning_rate" : 0.01,
        "verbosity" : -1
    }
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, num_boost_round = 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

In [54]:
## K-FOLD train

from sklearn import model_selection

train_X = train[cols_to_use]
test_X = test[cols_to_use]
train_y = train[target_col].values

pred_lgb = 0

kf = model_selection.KFold(n_splits = 5, random_state=2019, shuffle=True)
for trn_index, val_index in kf.split(train_X):
  trn_X, val_X = train_X.loc[trn_index,:], train_X.loc[val_index,:]
  trn_y, val_y = train_y[trn_index], train_y[val_index]
  pred_test_tmp, model, evals_result = run_lgb(trn_X, trn_y, val_X, val_y, test_X)
  pred_lgb += pred_test_tmp

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 168.9
[200]	valid_0's rmse: 165.631
[300]	valid_0's rmse: 163.713
[400]	valid_0's rmse: 162.891
[500]	valid_0's rmse: 162.431
[600]	valid_0's rmse: 162.413
[700]	valid_0's rmse: 162.221
[800]	valid_0's rmse: 162.018
[900]	valid_0's rmse: 161.918
Early stopping, best iteration is:
[892]	valid_0's rmse: 161.889


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: Qual_1, Qual_2, Qual_3, Qual_4, Qual_5, Qual_6, Qual_7, Qual_8, Qual_9, Qual_10

In [55]:
# Take average of 5 predictions and create submission file.

pred_lgb /= 5

test_lgb = df_test
test_lgb['Fees'] = pred_lgb
test_lgb.to_csv('E:\evaluation\p13\submission.csv')